# Submitting an experiment

In this notebook, we will use the Estimator object to submit an experiment to processing on a Azure-ML Compute Target.

An experiment is a process that can be tracked through its generated metrics.

In this particular case, we are using an external Python script to:

- get a dataset from Azure DataStorage
- create a machine learning pipeline for classification that includes:
    - grid search cross validation
    - logistic regression
    - decision trees

## Imports


In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator

In [2]:
# load ws
ws = Workspace.from_config()

# Get the dataset
data = ws.datasets.get('poker_ds')

# Dataset head
data.take(10).to_pandas_dataframe()

C:\Users\BJ571WQ\Envs\azure\lib\site-packages\azureml\dataprep\api\dataflow.py:722: UserWarning: Your pandas and pyarrow versions are incompatible. Please install pyarrow>=0.12.0 for improved performance of to_pandas_dataframe. You can ensure the correct version is installed by running: pip install azureml-dataprep[pandas].
  warnings.warn('Your pandas and pyarrow versions are incompatible. '


,Rank_1,Rank_2,Rank_3,Rank_4,Rank_5,class,Suit_1_2,Suit_1_3,Suit_1_4,Suit_2_2,...,Suit_2_4,Suit_3_2,Suit_3_3,Suit_3_4,Suit_4_2,Suit_4_3,Suit_4_4,Suit_5_2,Suit_5_3,Suit_5_4
0,11,13,10,12,1,9,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,12,11,13,10,1,9,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
2,10,11,1,13,12,9,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
3,1,13,12,11,10,9,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
4,2,4,5,3,6,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,9,12,10,11,13,8,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,2,3,4,5,8,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
7,5,6,9,7,8,8,0,1,0,0,...,0,0,1,0,0,1,0,0,1,0
8,1,4,2,3,5,8,0,0,1,0,...,1,0,0,1,0,0,1,0,0,1
9,1,1,9,5,3,1,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0


In [3]:
# get compute target and start it
cpu_cluster = ComputeTarget(workspace=ws, name='pc2')
cpu_cluster.start()
cpu_cluster.wait_for_completion(show_output=True)


Running


In [6]:
# Create an estimator
estimator = Estimator(
    source_directory='scripts',
    entry_script='experiment002.py',
    compute_target='pc2',
    conda_packages=['scikit-learn'],
    inputs=[data.as_named_input('poker')],
    )

In [7]:
# Submit
exp = Experiment(workspace=ws, name='PokerHand_Classification_ensamble')

# Run the experiment based on the estimator
run = exp.submit(config=estimator)
RunDetails(run).show()
run.wait_for_completion(show_output=True)


_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

RunId: PokerHand_Classification_ensamble_1591286585_6000473b
Web View: https://ml.azure.com/experiments/PokerHand_Classification_ensamble/runs/PokerHand_Classification_ensamble_1591286585_6000473b?wsid=/subscriptions/d8610c93-6c20-40ef-8ce5-281bf8b7f1d0/resourcegroups/erickfis-ml-rg/workspaces/poker-ws

Streaming azureml-logs/55_azureml-execution-tvmps_cbdabf560713eb9142515ebabda3f0ea1c1c5018f60ad0228a4b99c4cf72bc77_d.txt

2020-06-04T16:03:26Z Starting output-watcher...
2020-06-04T16:03:26Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
Login Succeeded
Using default tag: latest

Streaming azureml-logs/65_job_prep-tvmps_cbdabf560713eb9142515ebabda3f0ea1c1c5018f60ad0228a4b99c4cf72bc77_d.txt


Streaming azureml-logs/70_driver_log.txt

2020/06/04 16:03:34 Starting App Insight Logger for task:  runTaskLet
Entering context manager injector. Current time:2020-06-04T16:03:35.940409
Starting the daemon thread to refresh tokens in background for process with pid = 92
Entering Run 

ExperimentExecutionException: ExperimentExecutionException:
	Message: The output streaming for the run interrupted.
But the run is still executing on the compute target. 
Details for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "The output streaming for the run interrupted.\nBut the run is still executing on the compute target. \nDetails for canceling the run can be found here: https://aka.ms/aml-docs-cancel-run"
    }
}

In [8]:
run.fail()

## View Metrics

In [12]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
    print(key, metrics.get(key))
    print('\n')



logistic regression {'model name': 'logistic regression', 'train_score': 0.5011996246156969, 'test_score': 0.4963347994135679, 'parameters': {'C': 0.001, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 95276, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}}


decision tree {'model name': 'decision tree', 'train_score': 0.5365024368496494, 'test_score': 0.5517792882846861, 'parameters': {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 10, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'presort': 'deprecated', 'random_state': 95276, 'splitter': 'best'}}




## View Run History

In [13]:
exp = ws.experiments['PokerHand_Classification']
for logged_run in exp.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

Run ID: PokerHand_Classification_1591285052_7bd8684e
Run ID: PokerHand_Classification_1591284719_7634ace7
Run ID: PokerHand_Classification_1591284622_985c208b
Run ID: PokerHand_Classification_1591284438_cfce0cc0
Run ID: PokerHand_Classification_1591284325_501045a2
Run ID: PokerHand_Classification_1591021685_20be8e39
- logistic regression {'model name': 'logistic regression', 'train_score': 0.5009139756267729, 'test_score': 0.4964680794348927, 'parameters': {'C': 0.001, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': 95276, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}}
- decision tree {'model name': 'decision tree', 'train_score': 0.5365024368496494, 'test_score': 0.5517792882846861, 'parameters': {'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': None, 'max_leaf_nodes': None, 'm

## Stopping the compute target

In [9]:
for pc_name in ws.compute_targets:
    pc = ws.compute_targets[pc_name]
    pc.stop()

